In [3]:
sigma_at = 1.e5 #Pa
nu_e = 100.
omega_e = 0.6
e0 = 0.5

E_s = lambda sig, nu, om: nu * sigma_at * np.power(sig/sigma_at,om)

In [2]:
sigma_1 = np.linspace(sigma_at, 2.e5,1000) #Pa
sigma_2 = np.linspace(2.e5, 4.e5,1000) #Pa

NameError: name 'np' is not defined

In [5]:
def calc_strain(nu_e,omega_e):
    eps = np.array([0.])
    #Erstbelastung
    for i in range(1,len(sigma_1)):
        eps = np.append(eps,eps[-1]+(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e,omega_e))
    #Entlastung
    for i in range(len(sigma_1)-1,0,-1):
        eps = np.append(eps,eps[-1]-(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e*10,1))
    eps=np.append(eps,eps[-1])
    #Wiederbelastung
    for i in range(1,len(sigma_1)):
        eps = np.append(eps,eps[-1]+(sigma_1[i]-sigma_1[i-1])/E_s(sigma_1[i-1],nu_e*10,1))
    eps=np.append(eps,eps[-1])
    #Erstbelastung
    for i in range(1,len(sigma_2)):
        eps = np.append(eps,eps[-1]+(sigma_2[i]-sigma_2[i-1])/E_s(sigma_2[i-1],nu_e,omega_e))
    eps=np.append(eps,eps[-1]+(sigma_2[-1]-sigma_2[-2])/E_s(sigma_2[-2],nu_e,omega_e))
    return eps

In [5]:
def e(strain):
    return e0-strain*(1+e0)

In [7]:
def plot_interactive_curve():
    @interact(show_Ton = widgets.Checkbox(value=False, description='Ton'),
              show_Schluff = widgets.Checkbox(value=False, description='Schluff'),
              show_Sand_Kies = widgets.Checkbox(value=False, description='Sand und Kies')
    )
    def plot_curve(show_Ton,show_Schluff,show_Sand_Kies):
        
        eps_Ton = calc_strain(10,0.9)
        eps_Schluff = calc_strain(50,0.85)
        eps_Sand_Kies = calc_strain(400,0.65)        
        
        fig, ax = plt.subplots(ncols=2,figsize=(18,8))
        if (show_Ton):
            ax[0].plot(np.append(sigma_1,
                               np.append(np.append(np.flip(sigma_1),sigma_1),
                                         sigma_2))/1e3,
                  -eps_Ton*100)
        if (show_Schluff):
            ax[0].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              -eps_Schluff*100)
        if (show_Sand_Kies):
            ax[0].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              -eps_Sand_Kies*100)
        
        ticks =  ax[0].get_yticks()
        # set labels to absolute values and with integer representation
        ax[0].set_yticks(ticks)
        ax[0].set_yticklabels([round(abs(tick),3) for tick in ticks])
        ax[0].set_xlabel('$\\sigma$ / kPa')
        ax[0].set_ylabel('$\\epsilon$ / %')

        if (show_Ton):
            ax[1].plot(np.append(sigma_1,
                               np.append(np.append(np.flip(sigma_1),sigma_1),
                                         sigma_2))/1e3,
                  e(eps_Ton),label='Ton')
        if (show_Schluff):
            ax[1].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              e(eps_Schluff),label='Schluff')
        if (show_Sand_Kies):
            ax[1].plot(np.append(sigma_1,
                           np.append(np.append(np.flip(sigma_1),sigma_1),
                                     sigma_2))/1e3,
              e(eps_Sand_Kies),label='Sand und Kies')
        ax[1].set_xlabel('$\\sigma$ / kPa')
        ax[1].set_ylabel('$e$')
        ax[1].set_xscale('log')
        if (show_Ton or show_Schluff or show_Sand_Kies):
            ax[1].legend()
        plt.show;